In [ ]:
# Ejercicio 3.G

def get_max_metric_condicional(*args):
    argmax=np.argmax(args[0])

    results=[]
    for arg in args:
        results.append(arg[argmax])

    return results

x_train, x_validacion, x_test,y_train, y_validacion, y_test=get_datos()
y_score=new_model.predict(x_test).reshape(-1)
y_true=y_test

def get_threshold_informedness(y_true,y_score):
    rango_threshold=np.linspace(0,1,500)
    rango_informedness=[]
    rango_sensibilidad=[]
    rango_especificidad=[]    
    for threshold in rango_threshold:

        TP,TN,FP,FN=get_matriz_confusion(y_true,y_score,threshold=threshold)
        sensibilidad,especificidad,_,_,_=get_metrics(TP,TN,FP,FN) 
        rango_sensibilidad.append(sensibilidad) 
        rango_especificidad.append(especificidad)
        rango_informedness.append(sensibilidad+especificidad-1)

    return rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad


rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad=get_threshold_informedness(y_true,y_score)

figure=plt.figure(figsize=(6, 4)) 
axes=figure.add_subplot(1,1,1)
max_informedness,best_threshold,best_sensibilidad,best_especificidad=get_max_metric_condicional(rango_informedness,rango_threshold,rango_sensibilidad,rango_especificidad)
axes.plot(rango_threshold,rango_informedness,linestyle="solid",label=f"Sensibilidad+Especificidad-1")
axes.scatter(best_threshold,max_informedness,s=20,c="red",label=f"Max Informedness={max_informedness:.2f} con \nThr.={best_threshold:.2f} Sen.={best_sensibilidad:.2f} Esp={best_especificidad:.2f}",zorder=10)
axes_configure_for_metrics(axes,"Threshold vs Informedness","Threshold","Informedness")



print("Sensibilidad=",rango_sensibilidad[np.argmax(rango_informedness)])
print("Especificidad=",rango_especificidad[np.argmax(rango_informedness)])

In [ ]:
# Ejercicio 5.A

x_train, x_test, y_train, y_test=get_datos()

#### Entrenar los 2 modelos

capas=[30,64,32,16,8,1]
activation="ELU"
epochs=20
learning_rate=0.0001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
metrics=["Recall",specificity]

history1,model1=compile_fit(capas,activation,"sigmoid","binary_crossentropy",optimizer,epochs,x_train, x_test, y_train, y_test,metrics)


capas=[64,128,256,128,64,32,1]
activation="selu"
epochs=20
learning_rate=0.000005
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
metrics=["Recall",specificity]

history2,model2=compile_fit(capas,activation,"sigmoid","binary_crossentropy",optimizer,epochs,x_train, x_test, y_train, y_test,metrics)

###### Predecir la y_pred
y_score1=model1.predict(x_test).reshape(-1)
y_true1=y_test

y_score2=model2.predict(x_test).reshape(-1)
y_true2=y_test



In [ ]:
##### Mostar las gráficas durante el entrenamiento
figure=plt.figure(figsize=(6*2, 4*6)) 


axes=figure.add_subplot(6,2,1)
plot_metrics(axes,history1.history,"Loss","loss","Loss")
axes=figure.add_subplot(6,2,3)
plot_metrics(axes,history1.history,"Sensibilidad","recall","Sensibilidad")
axes=figure.add_subplot(6,2,5)
plot_metrics(axes,history1.history,"Especificidad","specificity","Especificidad")

axes=figure.add_subplot(6,2,2)
plot_metrics(axes,history2.history,"Loss","loss","Loss")
axes=figure.add_subplot(6,2,4)
plot_metrics(axes,history2.history,"Sensibilidad","recall","Sensibilidad")
axes=figure.add_subplot(6,2,6)
plot_metrics(axes,history2.history,"Especificidad","specificity","Especificidad")




###### Mostrar la matríz de confusión


TP,TN,FP,FN=get_matriz_confusion(y_true1,y_score1,threshold=0.5)
sensibilidad,especificidad,vpp,vpn,prevalencia=get_metrics(TP,TN,FP,FN)
axes=figure.add_subplot(6,2,7)
plot_matriz_confusion(axes,TP=TP,TN=TN,FP=FP,FN=FN,sensibilidad=sensibilidad,especificidad=especificidad,vpp=vpp,vpn=vpn,prevalencia=prevalencia)

TP,TN,FP,FN=get_matriz_confusion(y_true2,y_score2,threshold=0.5)
sensibilidad,especificidad,vpp,vpn,prevalencia=get_metrics(TP,TN,FP,FN)
axes=figure.add_subplot(6,2,8)
plot_matriz_confusion(axes,TP=TP,TN=TN,FP=FP,FN=FN,sensibilidad=sensibilidad,especificidad=especificidad,vpp=vpp,vpn=vpn,prevalencia=prevalencia)

####### Threshold vs Sensibiliad y Especificidad


rango_threshold,rango_sensibilidad,rango_especificidad=get_threshold_sensibilidad_especificidad(y_true1,y_score1)
axes=figure.add_subplot(6,2,9)
axes.plot(rango_threshold,rango_sensibilidad,linestyle="solid",label="Sensibilidad")
axes.plot(rango_threshold,rango_especificidad,linestyle="solid",label="Especificidad")
axes_configure_for_metrics(axes,"Threshold vs Sensibilidad y Especificidad","Threshold","Métricas")

rango_threshold,rango_sensibilidad,rango_especificidad=get_threshold_sensibilidad_especificidad(y_true2,y_score2)
axes=figure.add_subplot(6,2,10)
axes.plot(rango_threshold,rango_sensibilidad,linestyle="solid",label="Sensibilidad")
axes.plot(rango_threshold,rango_especificidad,linestyle="solid",label="Especificidad")
axes_configure_for_metrics(axes,"Threshold vs Sensibilidad y Especificidad","Threshold","Métricas")

####### Threshold vs Informedness


rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad=get_threshold_informedness(y_true1,y_score1)
axes=figure.add_subplot(6,2,11)
max_informedness1,best_threshold1,best_sensibilidad1,best_especificidad1=get_max_metric_condicional(rango_informedness,rango_threshold,rango_sensibilidad,rango_especificidad)
axes.plot(rango_threshold,rango_informedness,linestyle="solid",label=f"Sensibilidad+Especificidad-1")
axes.scatter(best_threshold1,max_informedness1,s=20,c="red",label=f"Max Informedness={max_informedness1:.2f} con \nThr.={best_threshold1:.2f} Sen.={best_sensibilidad1:.2f} Esp={best_especificidad1:.2f}",zorder=10)
axes_configure_for_metrics(axes,"Threshold vs Informedness","Threshold","Informedness")

rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad=get_threshold_informedness(y_true2,y_score2)
axes=figure.add_subplot(6,2,12)
max_informedness2,best_threshold2,best_sensibilidad2,best_especificidad2=get_max_metric_condicional(rango_informedness,rango_threshold,rango_sensibilidad,rango_especificidad)
axes.plot(rango_threshold,rango_informedness,linestyle="solid",label=f"Sensibilidad+Especificidad-1")
axes.scatter(best_threshold2,max_informedness2,s=20,c="red",label=f"Max Informedness={max_informedness2:.2f} con \nThr.={best_threshold2:.2f} Sen.={best_sensibilidad2:.2f} Esp={best_especificidad2:.2f}",zorder=10)
axes_configure_for_metrics(axes,"Threshold vs Informedness","Threshold","Informedness")


figure.tight_layout()

print("Me quedaría con la 2º red ya que tiene un mayor valor de Informedness")

In [ ]:
# Ejercicio 5.B
model2.save('modelo_definitivo.keras') 


In [ ]:
new_model=tf.keras.models.load_model('modelo_definitivo.keras',custom_objects={"specificity": specificity})

x_train, x_test, y_train, y_test=get_datos()
y_score=new_model.predict(x_test[[34,65,87,43,82,34,65,87,98,102,4]]).reshape(-1)
random.seed(12)


sensibilidad=best_sensibilidad2 #0.97
especificidad=best_especificidad2 #0.88
threshold=best_threshold2 #0.44

for index,predicho in enumerate(y_score):
    prevalencia=random.uniform(0.2, 0.8)
    if (predicho)>threshold:
        vpp_bayes=(sensibilidad*prevalencia)/(sensibilidad*prevalencia+(1-especificidad)*(1-prevalencia))
        print(f"El paciente Nº {index} es positivo con una probabilidad del {vpp_bayes*100:.0f}% ya que su prevalencia es del {prevalencia*100:.0f}%")
    else:
        vpn_bayes=(especificidad*(1-prevalencia))/(especificidad*(1-prevalencia)+(1-sensibilidad)*prevalencia)
        print(f"El paciente Nº {index} es negativo con una probabilidad del {vpn_bayes*100:.0f}% ya que su prevalencia es del {prevalencia*100:.0f}%")
    



In [ ]:
# Ejercicio 3.G

def get_max_metric_condicional(*args):
    argmax=np.argmax(args[0])

    results=[]
    for arg in args:
        results.append(arg[argmax])

    return results

x_train, x_validacion, x_test,y_train, y_validacion, y_test=get_datos()
y_score=new_model.predict(x_test).reshape(-1)
y_true=y_test

def get_threshold_informedness(y_true,y_score):
    rango_threshold=np.linspace(0,1,500)
    rango_informedness=[]
    rango_sensibilidad=[]
    rango_especificidad=[]    
    for threshold in rango_threshold:

        TP,TN,FP,FN=get_matriz_confusion(y_true,y_score,threshold=threshold)
        sensibilidad,especificidad,_,_,_=get_metrics(TP,TN,FP,FN) 
        rango_sensibilidad.append(sensibilidad) 
        rango_especificidad.append(especificidad)
        rango_informedness.append(sensibilidad+especificidad-1)

    return rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad


rango_threshold,rango_informedness,rango_sensibilidad,rango_especificidad=get_threshold_informedness(y_true,y_score)

figure=plt.figure(figsize=(6, 4)) 
axes=figure.add_subplot(1,1,1)
max_informedness,best_threshold,best_sensibilidad,best_especificidad=get_max_metric_condicional(rango_informedness,rango_threshold,rango_sensibilidad,rango_especificidad)
axes.plot(rango_threshold,rango_informedness,linestyle="solid",label=f"Sensibilidad+Especificidad-1")
axes.scatter(best_threshold,max_informedness,s=20,c="red",label=f"Max Informedness={max_informedness:.2f} con \nThr.={best_threshold:.2f} Sen.={best_sensibilidad:.2f} Esp={best_especificidad:.2f}",zorder=10)
axes_configure_labels(axes,"Threshold vs Sensibilidad y Especificidad","Threshold","Métricas")
axes_configure_axis_for_metrics(axes)



print("Sensibilidad=",rango_sensibilidad[np.argmax(rango_informedness)])
print("Especificidad=",rango_especificidad[np.argmax(rango_informedness)])

In [ ]:
def get_datos_entrada_a_predecir(numero_paciente=0):
    x=[]
    breast_cancer = load_breast_cancer()

    for index,nombre_dato in enumerate(breast_cancer.feature_names):
        max_value=max(breast_cancer.data[:,index])
        min_value=min(breast_cancer.data[:,index])
        defecto=breast_cancer.data[numero_paciente,index]

        valor=input(f"Introduce ({index+1}/{len(breast_cancer.feature_names)}) '{nombre_dato}' [{max_value:.2f},{min_value:.2f}], defecto:{defecto:.2f}:")
        if (valor==""):
            dato=defecto
        else:
            dato=float(valor)
        x.append(dato)

    return np.array(x)